In [1]:
import os

In [2]:
%pwd

'/home/laptop/projects/kidney_classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/laptop/projects/kidney_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_YAML_FILE_PATH, 
        params_filepath = PARAMS_YAML_FILE_PATH,
    ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
        )
        
        return prepare_base_model_config

In [9]:
# Update component
import os
import urllib.request as request
from zipfile import ZipFile
import torch.nn as nn
import torchvision.models as models

In [ ]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.model = None
    
    def get_base_model(self):
        # Load the pre-trained VGG16 model
        self.model = models.vgg16(weights=self.config.params_weights)
        # Remove the classification head (fully connected layers)
        self.model = nn.Sequential(*list(self.model.children())[:-1])
        # Save model
        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in list(model.children())[:-freeze_till]:
                model.trainable = False
        
        flattern_in = tf.keras.layers.Flattern()(model.output)
        prediction = tf.keras.layers.Dense(units=classes, activation="softmax")(flattern_in)
        
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction,
        )
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate)
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"],
        )
        
        full_model.summary()
        return full_model
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)